In [ ]:
%cd ..

# PreCode

In [ ]:
import os
from recfldgrn.datapoint import load_df_data_with_RecName, DataPoint
IDName = 'PID'; RANGE_SIZE = 10000

######################
rec_folder = 'data/ProcData/RecFolder'
####################

RecName_List = os.listdir(rec_folder)
print(RecName_List)
IDValue = 'P5'   # PhyID value
# print(data_folder)
Pat = DataPoint(IDName, IDValue, rec_folder, RANGE_SIZE)
# print(Pat)
dfx = Pat.get_df_rec('P')
df = load_df_data_with_RecName(rec_folder, 'P')
# dfx

# P@PatEcInfo-InfoGrn

## df_whole/row

### Settings

In [ ]:
###################
RecName_Chain = ['P'] 
RecTableName2FldColumns_Dict = {
    # The first should be in the level of RecName_Chain[-1]
    'P': ['PID', 'age', 'basicInfo'],
    'EC': ['DT_min', 'DT_max'],
    # TODO: to add the df_whole filter or dp filter, you may want to add the EC DT information. 
}
###################

########################
## Prediction Index Date
RecName = RecName_Chain[-1] # row level
SynFld = 'PECInfo' # This name must be special
GrnName = 'InfoGrn'
########################

###################
rec_folder = 'data/ProcData/RecFolder/'
###################

###################
reshape_fn_kwargs = {}
###################


### Load df_whole/row

In [ ]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
full_recfldgrn = f'{"-".join(RecName_Chain)}@{SynFld}-{GrnName}'

df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict, rec_folder)
# df_whole = get_df_individual_from_settings(RecName_Chain,  RecTableName2FldColumns_Dict, Pat,)
row = df_whole.iloc[0]

### Filter df_whole/row (optional)

In [ ]:
import pandas as pd

for Rec in RecTableName2FldColumns_Dict:
    print(f'\n{Rec} =======')
    s = df_whole[Rec].apply(lambda x: len(x) if type(x) == type(pd.DataFrame()) else 'Missing')
    print(s.value_counts().sort_index())

## grain_tkn_fn

### Function Development

In [ ]:
import inspect
from recfldgrn.graintools import func_convert_Num_factory

def grain_tkn_fn(row):
    import itertools
    
    buffer_dict = {}
    
    # Patient Part: 
    df_recinput = row['P'][['age', 'basicInfo']] # this special because xxx.
    
    # P@Age
    fld = 'age'
    start, end, Min, Max, scale = [18, 80, 1, 120, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = df_recinput[fld].iloc[0]
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}_{i}' for i in results[0]]
    li_value_wgt = results[1]
    # print(len(li_value_tkn), len(li_value_wgt))
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # P@basicInfo
    fld = 'basicInfo'
    basicInfo_series = df_recinput[fld]# .mean()
    value = basicInfo_series.iloc[0]
    li_value_tkn = value.split('&')
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # EC Part
    df_recinput = row['EC'][['DT_min', 'DT_max']]
    
    # P@ECNum
    fld = 'ecNum'
    value = len(df_recinput)
    li_value_tkn = [f'{fld}_{value}' ]
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # P@ecDays
    fld = 'ecDays'
    start, end, Min, Max, scale = [0, 100, 0, 500, 7]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = (df_recinput['DT_max'] - df_recinput['DT_min']).mean().total_seconds() / 60 / 60 / 24
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}{i}' for i in results[0]]
    li_value_wgt = results[1]
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # summary
    output = {}
    for i in ['key', 'wgt', 'tpc']:
        # output[i] = sum(, [])
        output[i.split('_')[-1]] = list(itertools.chain(*[d[i] for fld, d in buffer_dict.items()]))
        
    # key: tkn
    # value: wgt
    # fld: hypertype?
    return output

func_convert_Num_factory.fn_string = inspect.getsource(func_convert_Num_factory)
grain_tkn_fn.fn_string = inspect.getsource(grain_tkn_fn)
# print(grain_str_fn_string)

### Test

In [ ]:
output = grain_tkn_fn(row)
for name, grn_list in output.items():
    print('\n')
    print(name)
    print(grn_list)
    print(len(grn_list))

In [ ]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]]
df_fld

## Save PyFile

### Prepare Variables

In [ ]:
string_variables = [RecName, SynFld, GrnName, rec_folder]
iterative_variables = [RecName_Chain, RecTableName2FldColumns_Dict, reshape_fn_kwargs]
fn_variables = [grain_tkn_fn, func_convert_Num_factory]
prefix = ['import pandas as pd', 'import numpy as np', 'import itertools']

### Save to PyFile

In [ ]:
from recfldgrn.loadtools import retrieve_name, convert_variables_to_pystirng

python_strings  = convert_variables_to_pystirng(string_variables, iterative_variables, fn_variables, prefix)
Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
print(meta_file)
with open(meta_file, 'w') as f: 
    f.write(python_strings)

# Application

In [ ]:
# restart
%cd ..

## Settings

In [ ]:
##########
rec_folder = 'data/ProcData/RecFolder/'
SynFld = 'PECInfo' # ----> get its MetaDict
##########


# filtering hyperparameters
######### get df_whole: obsind, predind, and futind
PpredTable = 'xxx'
PpredColum = 'xxx'
Range_List = ['Bf2y']
#########

## Load FldGrn PyFile

In [ ]:
import os
import pandas as pd
from recfldgrn.loadtools import load_module_variables

Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
module = load_module_variables(meta_file)
MetaDict = module.MetaDict

RecName = MetaDict['RecName']#  'P'
RecName_Chain = MetaDict['RecName_Chain'] # ['P']
# RecName_Chain_to_RecName = MetaDict['RecName_Chain_to_RecName'] # {} # Can be updated further. 
RecTableName2FldColumns_Dict = MetaDict['RecTableName2FldColumns_Dict']
# SynFld = MetaDict['SynFld'] # 'PECINfo'
GrnName = MetaDict['GrnName'] # 'InfoGrn'
# sfx_info = MetaDict['sfx_info']
# rec_folder = MetaDict['rec_folder'] # 'data/ProcData/RecFolder/'
reshape_fn_kwargs = MetaDict['reshape_fn_kwargs']
grain_tkn_fn = MetaDict['grain_tkn_fn']

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
recfldgrn

##  Load & Filter df_whole/row

In [ ]:
RecTableName2FldColumns_Dict = RecTableName2FldColumns_Dict # TODO, to update.
# RecTableName2FldColumns_Dict[PpredTable] = [PpredColum]

In [ ]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings
df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict,  rec_folder)

In [ ]:
## filtering

## Generate RecFldGrn

In [ ]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]].reset_index(drop = True)
df_fld

## Save FldGrn Data (Optional)

# Dataset Construction

## Track 1: df_table (ml)

In [ ]:
# Your input here: you can have multiple df_fld, actually. 
print(df_fld.shape)
df_fld.head()

### prepare df_ml

In [ ]:
# TODO: this need further operations
df_ml = df_fld.copy()

In [ ]:
df_value = pd.DataFrame(df_ml[recfldgrn].apply(lambda x: dict(zip(x['key'], x['wgt']))).to_list())
df_table = pd.concat([df_ml[[R+'ID' for R in RecName_Chain]], df_value], axis = 1)
df_table

## Track 2: df_tensor

### prepare df_fld

In [ ]:
# Your input here: you can have multiple df_fld, actually. 
print(df_fld.shape)
df_fld.head()

### get df_tensor

In [ ]:
# TODO: this need further operations
df_tensor = df_fld.copy()

In [ ]:
# we can save it for the future. 
for i in ['key', 'tpc', 'wgt']: 
    df_tensor[f'{recfldgrn}_{i}'] = df_tensor[recfldgrn].apply(lambda x: x[i])
    
df_tensor = df_tensor.drop(columns = [recfldgrn])
df_tensor

### get idx and EmbedDict

In [ ]:
import pandas as pd 
from recfldgrn.graintools import generate_grain_vocab_info

def generate_EmbedDict_from_processed_df_whole(df_whole, recfldgrn, sfx_list = ['key', 'tpc']):
    EmbedDict = {}
    
    for sfx in sfx_list:
        embed_type = 'CateEmbed'
        recfldgrn_sfx = f'{recfldgrn}_{sfx}'
        # print(recfldgrn_sfx)
        s = df_whole[recfldgrn_sfx] 
        Vocab = generate_grain_vocab_info(s) 
        v2idx = Vocab['v2idx']
        vocab_size = len(v2idx)
        # print(v2idx)
        d = {'embed_type':embed_type, 
             'recfldgrn_sfx':recfldgrn_sfx,
             'vocab_size': vocab_size,
             'Vocab': Vocab}
        
        df_whole[f'{recfldgrn}_{sfx}idx'] =  df_whole[f'{recfldgrn}_{sfx}'].apply(lambda x: [v2idx[i] for i in x])
        EmbedDict[sfx] = d
        
    return df_whole, EmbedDict


In [ ]:
df_tensor, EmbedDict = generate_EmbedDict_from_processed_df_whole(df_tensor, recfldgrn)
df_tensor.head()

In [ ]:
print(EmbedDict)

### Adjust to dp Level

In [ ]:
tensor_cols = [i for i in df_tensor.columns if 'idx' == i[-3:] or 'wgt' == i[-3:]]
final_cols = [R + 'ID' for R in RecName_Chain] + tensor_cols
df_tensor = df_tensor[final_cols]
df_tensor

In [ ]:
from recfldgrn.graintools import get_highorder_input_idx
from functools import reduce


df_list = []
prefix_ids = [R + 'ID' for R in RecName_Chain][:-1]
focal_ids = [R + 'ID' for R in RecName_Chain][-1:]
for recfldgrn_sfx in tensor_cols:
    df_p = get_highorder_input_idx(df_tensor, recfldgrn_sfx, prefix_ids, focal_ids)
    df_list.append(df_p)
    
# Merge the dataframes in the list using reduce
df_tensor_fnl = reduce(lambda left, right: pd.merge(left, right), df_list)
df_tensor_fnl

In [ ]:
prefix_recs = [i.replace('ID', '') for i in prefix_ids]
full_recfldgrn = '-'.join(prefix_recs) + '-' + recfldgrn if len(prefix_recs) > 0 else recfldgrn
full_recfldgrn

### Save to Dataset

In [ ]:
# from recfldgrn.datapoint import write_df_to_folders, RANGE_SIZE

# fldgrnTensor_folder = f'data/ProcData/FldGrnTensor/{full_recfldgrn}'
# if not os.path.exists(fldgrnTensor_folder): os.makedirs(fldgrnTensor_folder)

# print(recfldgrn)
# write_df_to_folders(recfldgrn, fldgrnTensor_folder, df_tensor_fnl, IDName)